In [2]:
import numpy as np
import openai
import pandas as pd
import os
import sys
import time

# Add the path to the constants file to the system path
sys.path.append('../../../')
from constants import *
from evaluation_utils import *
from path_utils import *
from ChatCompletion_OpenAI_API import *

# OpenAI API Key
openai.api_key = OPENAI_API_KEY

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
# source code folder path
rec_sys_dir = get_rec_sys_directory()
print(f"Rec-sys directory: {rec_sys_dir}")

# data folder path
DATA_DIR = os.path.join(rec_sys_dir, '../data')
print(f"Data directory: {DATA_DIR}")

# data path
data_path = os.path.join(DATA_DIR, 'ml-1m/merged_data.dat')
print(f'Data path: {data_path}')

# output
ZERO_SHOT_SAVE_PATH = os.path.join(DATA_DIR, 'ml-1m/output/title_zero_shot.dat')
print(f'Data path: {ZERO_SHOT_SAVE_PATH}')

ZERO_SHOT_RERUN_PATH = os.path.join(DATA_DIR, 'ml-1m/output/rerun_title_zero_shot.dat')
print(f'Data path: {ZERO_SHOT_RERUN_PATH}')

# few shot save path
FEW_SHOT_1_OBS_SAVE_PATH = os.path.join(DATA_DIR, 'ml-1m/output/title_1_test_predictions_few_shot.csv')
print(f'Few shot save path: {FEW_SHOT_1_OBS_SAVE_PATH}')


# few shot save path
FEW_SHOT_1_OBS_RERUN_PATH = os.path.join(DATA_DIR, 'ml-1m/output/title_1_test_predictions_few_shot.csv')
print(f'Few shot save path: {FEW_SHOT_1_OBS_SAVE_PATH}')

Rec-sys directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/notebook
Data directory: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/notebook/../data
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/notebook/../data/ml-1m/merged_data.dat
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/notebook/../data/ml-1m/output/title_zero_shot.dat
Data path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/notebook/../data/ml-1m/output/rerun_title_zero_shot.dat
Few shot save path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/notebook/../data/ml-1m/output/title_1_test_predictions_few_shot.csv
Few shot save path: /Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/notebook/../data/ml-1m/output/title_1_test_predictions_few_shot.csv


# Data Overview

In [4]:

# Read the data
data = pd.read_csv(data_path)

# get statistic and first few data of NUM_SAMPLES rows
data.info()
data.head(NUM_EXAMPLES)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   UserID      1000209 non-null  int64 
 1   MovieID     1000209 non-null  int64 
 2   Rating      1000209 non-null  int64 
 3   Timestamp   1000209 non-null  int64 
 4   Gender      1000209 non-null  object
 5   Age         1000209 non-null  int64 
 6   Occupation  1000209 non-null  int64 
 7   Zip-code    1000209 non-null  object
 8   Title       1000209 non-null  object
 9   Genres      1000209 non-null  object
dtypes: int64(6), object(4)
memory usage: 76.3+ MB


,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


# Zero-shot (OpenAI API)

In [10]:
%%time

predict_ratings_zero_shot_and_save(data,
                                       columns_for_prediction=['Title'],
                                       user_column_name='UserID',
                                       title_column_name='Title',
                                       asin_column_name='MovieID',
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path=ZERO_SHOT_SAVE_PATH)

Constructed Prompt for zero-shot approach:

The prompt:
**********
How will user rate this Title: One Flew Over the Cuckoo's Nest (1975)? (1 being lowest and 5 being highest) Attention! Just give me back the exact number as a result, and you don't need a lot of text.

Based on the above information, please predict user's rating for the product: (1 being lowest and 5 being highest, The output should be like: (x stars, xx%), do not explain the reason.)
**********



System Fingerprint: fp_eeff13170a

API call response: "4"
Extracted rating: 4.0


KeyError: 'rating'

In [ ]:
import pandas as pd

# Read the data
data = pd.read_csv(ZERO_SHOT_SAVE_PATH)

# Display the original data types
# print("Original Data Types:")
# print(data.dtypes)
# print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
data['is_rating_float'] = pd.to_numeric(data['predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = data[data['is_rating_float'] == False]

# Number of rows with non-float ratings
print(f"Number of rows with non-float ratings: {len(non_float_ratings)}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


In [11]:
%%time

data = pd.read_csv(ZERO_SHOT_SAVE_PATH)

# Rerun predictions for failed cases and save the updated data
rerun_save_path = os.path.join(DATA_DIR, 'movie-ml-latest-small/output/rerun_title_large_predictions_zero_shot.csv')
columns_for_prediction = ['title']
updated_data = rerun_failed_zero_shot_predictions(data, ZERO_SHOT_SAVE_PATH, rerun_save_path, columns_for_prediction, PAUSE_EVERY_N_USERS, SLEEP_TIME)

# Remove rows with non-float ratings and save the cleaned data
cleaned_data = updated_data[pd.to_numeric(updated_data['predicted_rating'], errors='coerce').notna()]
cleaned_data.to_csv(ZERO_SHOT_SAVE_PATH, index=False)

# Evaluate the model predictions
evaluate_model_predictions_rmse_mae(ZERO_SHOT_SAVE_PATH, NUM_EXAMPLES, 'actual_rating', 'predicted_rating')


FileNotFoundError: [Errno 2] No such file or directory: '/Users/tnathu-ai/VSCode/recommender-system/recommender-system-openAI/rec-sys/notebook/../data/ml-1m/output/title_zero_shot.dat'

# Few-shot (OpenAI API)


+ For each user, we'll use 4 of their ratings as training data to predict ratings for the rest of their products. Finally, we'll evaluate the predictions against the actual ratings to calculate the overall RMSE and MAE.

+ The rating_history_str now includes both the title and the review text for each of the training data rows

# 1 observation per reviewer - Few-shot OpenAI

In [9]:
%%time

predict_ratings_few_shot_and_save(data,
                                      columns_for_training=['Title'],
                                       columns_for_prediction=['Title'],
                                       user_column_name='UserID',
                                       asin_column_name='MovieId', 
                                       obs_per_user=1,
                                       pause_every_n_users=PAUSE_EVERY_N_USERS,
                                       sleep_time=SLEEP_TIME,
                                       save_path=FEW_SHOT_1_OBS_SAVE_PATH)


TypeError: predict_ratings_few_shot_and_save() missing 1 required positional argument: 'columns_for_training'

In [ ]:
import pandas as pd

# Read the data
data = pd.read_csv(FEW_SHOT_1_OBS_SAVE_PATH)

# Display the original data types
# print("Original Data Types:")
# print(data.dtypes)
# print("\n")

# Attempt to convert ratings to float and add a flag for conversion failure
data['is_rating_float'] = pd.to_numeric(data['predicted_rating'], errors='coerce').notna()

# Filter rows where ratings are not float
non_float_ratings = data[data['is_rating_float'] == False]

# total number of rows with non-float ratings
print(f"Total number of rows with non-float ratings: {len(non_float_ratings)}")

# Display rows with non-float ratings
print("Rows with non-float ratings:")
non_float_ratings.head(3)


In [ ]:
evaluate_model_predictions_rmse_mae(
    data_path=FEW_SHOT_1_OBS_SAVE_PATH,
    num_examples=NUM_EXAMPLES,
    actual_ratings_column='actual_rating',
    predicted_ratings_column='predicted_rating'
)

# Limitations:

The model might not fully understand the nuanced relationships between products based on titles alone. Additional context or features might be needed for more accurate predictions.
This approach might be computationally expensive and slower than traditional matrix factorization or deep learning-based recommendation models, especially for a large number of users.

# References

+ https://platform.openai.com/docs/api-reference/authentication